<a href="https://colab.research.google.com/github/Jake-Jasper/AI-Capstone-Project/blob/main/Generate_text_and_rerank_from_local_docs_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Generative AI - Aumented with documents

Incorporate multiple docs

In [ ]:
!pip install -Uq gpt4all sentence-transformers cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 919.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import util, SentenceTransformer
import cohere
from gpt4all import GPT4All
import sqlite3
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
co = cohere.Client("5xvQOPxJqxpUX03AQRONJ1MJTG9knBBTCMHyFjFg")

In [ ]:
sent_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2');

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#model = GPT4All(model_path = "e:\Downloads", model_name="Nous-Hermes-2-Mistral-7B-DPO.Q4_0.gguf")
## Not sure where collab stores models. You can upload a model to your drive and then
## load it e.g. above
gen_model = GPT4All("orca-mini-3b-gguf2-q4_0.gguf")

In [ ]:
question = "How should I store Bananas?"

In [ ]:
## 100 tokens is ~ 3 sentences
DB = "/content/drive/MyDrive/12-4-24-all-mini-token-100.db"

def find_top_k_relevance(question=["No question"], model=None, n=20, db=DB):
  q = model.encode(question)
  scores = {}
  conn = sqlite3.connect(db)

# Create a cursor object
  cursor = conn.cursor()
  # load row by row
  cursor.execute('SELECT * FROM documents')
  for row in cursor:
    scores[row[0]] = util.pytorch_cos_sim(q, np.frombuffer(row[4], dtype=np.float32)).numpy()

  return dict(sorted(scores.items(), key = lambda x: x[1], reverse = True)[:n])


top_k = find_top_k_relevance(question =["What temperature should I keep Milk?"], model=sent_model)

In [ ]:
def get_relevant_docs(top_k):
    top_docs = list(top_k.keys())

    conn = sqlite3.connect(DB)
    docs = []
    # Create a cursor object
    cursor = conn.cursor()
    # load row by row
    # have to do the comprehension to unpakc inside and f-string
    # this doesn't get then in order
    for i in top_docs:
        docs.append(cursor.execute(f"""SELECT sentence FROM documents
                    WHERE id = {i}""").fetchone())

    docs = [i[0] for i in docs]
    results = co.rerank(model = "rerank-english-v3.0", query=q, documents=docs, top_n=4, return_documents=True)
    relevant_ids = []
    # relevant doc ids
    rel_documents = []
    # texts for prompt
    prompt_candidates = []

    for idx, r in enumerate(results.results):
        # only get the best ones..cohere seems to rank them higher than with cosine so 0.4 should be good
        if r.relevance_score > 0.7:
            relevant_ids.append(r.index)

    db_ids = [top_docs[i] for i in relevant_ids]

    conn = sqlite3.connect(DB)
    cursor = conn.cursor()

    for i in db_ids:
        x = cursor.execute(f"""SELECT * FROM documents
                        WHERE id = {i}""").fetchone()

        rel_documents.append(x[2])
        prompt_candidates.append(x[3])

    conn.close()

    return set(rel_documents), prompt_candidates

rel_docs, prompt_candidates = get_relevant_docs(top_k)

In [ ]:
prompt_candidates

[]

In [ ]:
# Join relevant text together
retrieved = "\n".join(i for i in prompt_candidates)

### Need to have some checking so that the context length doesn't get too long the max it can currently be is 4 x 100 tokens ish

In [ ]:
retrieved

'. Visual quality was maintained longer where the fruit was kept in polyethylene bags. It is not appropriate to store bananas in the fridge as they are chilling-sensitive, and their skins become blackened, but the coolest place in the home is the best storage location and the fruit should be kept in bags. Bananas sold in the UK are mostly sourced from Central and South America. They are harvested while still in a pre-climacteric (green) condition and shipped to the UK where'

In [ ]:
# modified from here
q = "How should I store Bananas?"
x = f"""### System:
You are an sophisticated AI assistant that helps to answer questions as best you can.

### User:
{q}

### System:
Based on the retrieved information from the PDF, here are the relevant excerpts:

{retrieved}

Please answer the user's question, integrating insights from these excerpts and your general knowledge. Limit your answer to Three sentences.

### Response:
"""

In [ ]:
response = gen_model.generate(prompt="How Should I store Bananas?", temp=0)
print(response)


Bananas are a tropical fruit that require warm, humid conditions to grow and ripen properly. They should be stored in a cool, dry place away from direct sunlight. Here's how you can store bananas:
1. Store them at room temperature or in the refrigerator. Avoid storing them near fruits with strong odors as they may spoil faster.
2. Use a banana storage bag or wrap to keep them fresh for up to two weeks. You can also use a breathable produce wrap or a reusable container filled with water to help keep them moist.
3. If you're not going to eat the bananas within a few days, consider freezing them. Remove any damaged or moldy fruit before freezing and defrost them in the refrigerator.


In [ ]:
response = gen_model.generate(prompt=x, temp=0)
print(response)

 To store bananas, it is recommended to keep them in polyethylene bags at room temperature to retain moisture content and visual quality for longer, up to 3 days. Bananas stored at 22°C lost their visual quality within 4 days, which may result in citizens wasting the fruit even though the internal (eating) quality may be acceptable. Bananas kept at 15°C remained visually acceptable for 7 - 14 days, which endorses the advice provided by retailers to maintain cool conditions.


In [ ]:
# question list
Questions = [
        "How should I store bananas?",
        "What temperature should I keep milk?",
        "How do I increase shelf life of lettuce?",
        "How does selling fresh produce loose, affect waste?",
        "What is a best-before date for?",
        "What is a use-by date for?",
        "How does removing date labels help?",
        "What are the reasons to not participate in a food waste scheme?",
        "What is the UN's sustainable development goal with respect to food?",
        "Which document relates to primary producers and food waste?",
        "Which document relates to retailers and food waste?",
        "What is the most common use for digestate?",
        "What affects the costs of valorisation?",
        "How many tonnes of food waste are there each year?",
        "What is the benefit of caddy liners?",
        "How to prevent food loss?",
        "How to prevent food waste?",
        "How does composting affect food waste?",
        "What are the main drivers of food waste?",
        "Online shopping and food waste",
        "What is involved with Courthold 2030?",
        "How do you measure in field food waste?"
]

In [ ]:
### Store results in dataframe
cols = ["model", "db", "Question", "retrieved", "default_ans", "context_ans"]

df = pd.DataFrame(columns = cols)

In [ ]:
## models
# mistral-7b-openorca.gguf2.Q4_0.gguf - best overall chat model and commercially licensed
gen_model = GPT4All("gpt4all-falcon-newbpe-q4_0.gguf")

Downloading: 100%|██████████| 4.21G/4.21G [01:27<00:00, 48.0MiB/s]
Verifying: 100%|██████████| 4.21G/4.21G [00:28<00:00, 150MiB/s]


In [ ]:
# orca-mini - orca-mini-3b-gguf2-q4_0.gguf
# nous-hermes-llama2-13b.Q4_0.gguf next
gen_mod = "gpt4all-falcon"

for q in tqdm(Questions):
    # get top docs from db
    top_k = find_top_k_relevance(question =q, model=sent_model)
    # re rank docs and get best docs
    rel_docs, prompt_candidates = get_relevant_docs(top_k)
    # if no relevant docs
    # Join relevant text together
    retrieved = "\n".join(i for i in prompt_candidates)
    if retrieved == "" or retrieved == "\n":
        retrieved = "No relevant documents found"

    default_ans = gen_model.generate(prompt=q, temp=0)

    # prompt
    x = f"""### System:
            You are an sophisticated AI assistant that helps to answer questions as best you can.

            ### User:
            {q}

            ### System:
            Based on the retrieved information from the PDF, here are the relevant excerpts:

            {retrieved}

            Please answer the user's question, integrating insights from these excerpts and your general knowledge. Limit your answer to Three sentences.

            ### Response:
            """
    # answer with context
    context_ans = gen_model.generate(prompt=x, temp=0)
    # append to database
    df.loc[len(df.index)] = [gen_mod, DB, q,retrieved , default_ans, context_ans]



100%|██████████| 22/22 [3:21:08<00:00, 548.56s/it]


In [ ]:
df

,model,db,Question,retrieved,default_ans,context_ans
0,mistral-7b-openorca,/content/drive/MyDrive/12-4-24-all-mini-token-...,How should I store bananas?,. Visual quality was maintained longer where t...,\n,"To store bananas properly, keep them in a coo..."
1,mistral-7b-openorca,/content/drive/MyDrive/12-4-24-all-mini-token-...,What temperature should I keep milk?,78%+130%+28%+26%051015202530 Optimal fridge (4...,\n,"To keep milk at its best quality, it should b..."
2,mistral-7b-openorca,/content/drive/MyDrive/12-4-24-all-mini-token-...,How do I increase shelf life of lettuce?,extending shelf-life through packaging. Waitr...,\n,"To increase the shelf life of lettuce, you ca..."
3,mistral-7b-openorca,/content/drive/MyDrive/12-4-24-all-mini-token-...,"How does selling fresh produce loose, affect w...",OVERVIEW OF RECOMMENDATIONSRetailers and suppl...,\n,Selling fresh produce loose can potentially r...
4,mistral-7b-openorca,/content/drive/MyDrive/12-4-24-all-mini-token-...,What is a best-before date for?,CThese results indicate that the Best Before d...,\n,A best-before date is a label on products ind...
5,mistral-7b-openorca,/content/drive/MyDrive/12-4-24-all-mini-token-...,What is a use-by date for?,No documents found,\n,A use-by date is a label on food products tha...
6,mistral-7b-openorca,/content/drive/MyDrive/12-4-24-all-mini-token-...,How does removing date labels help?,date codes – could help to reduce household f...,\n,Removing date labels on uncut fresh produce i...
7,mistral-7b-openorca,/content/drive/MyDrive/12-4-24-all-mini-token-...,What are the reasons to not participate in a f...,It summarises the following points: - Main r...,\n,1) The main reason people do not participate i...
8,mistral-7b-openorca,/content/drive/MyDrive/12-4-24-all-mini-token-...,What is the UN's sustainable development goal ...,"auld 2030 targets, and the UN's Sustainable De...",\n,The UN's Sustainable Development Goal (SDG) 1...
9,mistral-7b-openorca,/content/drive/MyDrive/12-4-24-all-mini-token-...,Which document relates to primary producers an...,The document outlines actions for primary prod...,\n,The document that relates to primary producer...


In [ ]:
df.to_csv("/content/drive/MyDrive/gpt4all-falcon-100token-rerank.csv")